In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import sqlite3
import sqlalchemy

In [ ]:
# Создайте базу данных sqlite, можно использовать sqlalchemy или pandas, или любой другой инструмент

CON = sqlite3.connect('currency.db', isolation_level=None)

In [ ]:
# Напишите функцию которая принимает на вход дату
# и выгружает курс валюты EUR к USD за этот день
# Функция должна вернуть список словарей или можно использовать можно использоват pandas.read_csv()

# Для этого воспользуйтесь https://exchangerate.host/#/
# и методом Timeseries Endpoint! Строго им. Где start_date = end_date = date
# Используйте параметры start_date, end_date, base=EUR, symbols=USD, format=csv

def extract_currency(date):
  url = 'https://api.exchangerate.host/timeseries?start_date=' + str(date) + '&end_date=' + str(date) + "&base='EUR'" + "&symbols=USD" + '&format=csv'

  return pd.read_csv(url)
# extract_currency('2021-01-01')

In [ ]:
# Напишите функцию которая скачивает данные с гитхаба
# Функция принимает на вход дату
# Функция должна вернуть список словарей или можно использовать можно использоват pandas.read_csv()

# ссылка на гитхаб с файлами - https://github.com/dm-novikov/stepik_airflow_course/tree/main/data_new

def extract_data(date):
  url = 'https://raw.githubusercontent.com/dm-novikov/stepik_airflow_course/main/data_new/' + str(date) + '.csv'
  return pd.read_csv(url)

# extract_data('2021-01-01')

In [ ]:
# Напишите функцию которая принимает на вход список словарей
# и загружает в табличку в sqlite, можно использоват pandas.to_sql()

def insert_to_db(data, table_name, conn):
  data.to_sql(table_name, con=conn, if_exists='replace', index=False)



In [ ]:
# Напишите функцию которая выполнит SQL запрос к базу данных
# Здесь удобно использовать sqlalchemy или https://pythonru.com/biblioteki/vvedenie-v-sqlite-python
# Если выполняется select ... то функция должна возвращать результат (не None)
# Если используется внутренняя операция например truncate table, insert into select ... то возвращает None
# data = sql_query(sql="select * from table") -> [(1, 2, 3), (4, 5, 6) ...]
# sql_query("truncate table") -> None

def sql_query(sql, conn):
  df = None
  cursor = conn.cursor()
  query = cursor.execute(sql)
  if query.description:
    df = pd.DataFrame.from_records(data = query.fetchall(), columns = [column[0] for column in query.description])
  cursor.close()
  return df

In [ ]:
# Напишите функцию которая отправит полученный отчет на почту
# Для этого можете воспользоваться кодом который был написан ранее (на уроке)
# обернув его в вашу функцию

def send_report(data, email):
  pass

In [ ]:
# Запустите ваш код в функции main

# Напишите генерацию дат, так чтобы у вас получился список
# 2021-01-02, 2021-01-03 ... etc
# Нужны даты с 2021-01-01 по 2021-01-04
from datetime import datetime

dates_list = [d.strftime('%Y-%m-%d') for d in pd.date_range('2021-01-01', '2021-01-04')]
def main(date, email, conn):
  
  # Выгружаем данные по валютам и из источника
  currency = extract_currency(date)
  data = extract_data(date)

  # Создайте необходимые таблицы если это нужно
  # Вставляем данные в БД
  # У вас долнжо получиться 2 таблицы CURRENCY, DATA
  insert_to_db(currency, 'currency', conn)
  insert_to_db(data, 'data', conn)

  # Объединение данных в отдельную таблицу JOIN_DATA
  # CURRENCY, DATA объединить через JOIN по дате и валюте
  # Используйте insert into select ... inner join ...
  # И соответственно создать таблицу для этого
  join = sql_query("""select c.date
                                  ,d.currency	
                                  ,c.code
                                  ,c.rate	
                                  ,c.start_date	
                                  ,c.end_date 
                                  ,d.value	
                             from currency c
                            inner join data d
                                    on d.currency = replace(c.base,'''', '')
                                   and d.date = c.date
            """, conn)
  insert_to_db(join, 'join_data', conn)

  # Очистка временных таблиц CURRENCY, DATA
  # Это те таблицы куда произошла вставка данных
  # Используйте truncate
  sql_query("""DELETE FROM currency
            """, conn)
  sql_query("""DELETE FROM data
            """, conn)
  # Получение данных из таблицы JOIN_DATA за конкретный день
  # Используйте date при формировании запроса select * from ... where date ='...'
  report = sql_query(f"select * from join_data where date='{date}' and code='USD'", conn) 
  print(report)

# Пройдемся по списку дат и выполним скрипт
for date in dates_list:
  main(date, 'email', CON)

         date currency code      rate  start_date    end_date  value
0  2021-01-01      EUR  USD  1,217582  2021-01-01  2021-01-01     23
1  2021-01-01      EUR  USD  1,217582  2021-01-01  2021-01-01     24
2  2021-01-01      EUR  USD  1,217582  2021-01-01  2021-01-01     38
3  2021-01-01      EUR  USD  1,217582  2021-01-01  2021-01-01     42
4  2021-01-01      EUR  USD  1,217582  2021-01-01  2021-01-01     48
5  2021-01-01      EUR  USD  1,217582  2021-01-01  2021-01-01     65
6  2021-01-01      EUR  USD  1,217582  2021-01-01  2021-01-01     74
7  2021-01-01      EUR  USD  1,217582  2021-01-01  2021-01-01     74
8  2021-01-01      EUR  USD  1,217582  2021-01-01  2021-01-01     86
         date currency code    rate  start_date    end_date  value
0  2021-01-02      EUR  USD  1,2135  2021-01-02  2021-01-02      3
1  2021-01-02      EUR  USD  1,2135  2021-01-02  2021-01-02     27
2  2021-01-02      EUR  USD  1,2135  2021-01-02  2021-01-02     36
3  2021-01-02      EUR  USD  1,2135  2021-

In [ ]:
# # чтобы првоерить решение можете обратиться к вашей базе данных таким образом
# %load_ext sql
# %config SqlMagic.feedback=False 
# %config SqlMagic.autopandas=True
# %sql sqlite:////currency.db
# %sql select * from join_data